# **EduSense: Production-Ready 2-Stage Architecture**

**King Khalid University - Graduation Project 2025**

---

## Architecture Overview

### **Stage A: Offline Feature Extraction** (Run Once)
```
Video → YOLO (face detection) → ViT (frozen features) → Save .npy embeddings
```

### **Stage B: Training & Inference**
```
Load .npy → LSTM (temporal modeling) → KAN (classification) → Confusion Score
```

**Key Improvements:**
- ✅ Feature extraction decoupled from training
- ✅ ViT frozen (no backprop through transformer)
- ✅ Fast training (no video I/O)
- ✅ Easy deployment (embeddings can be pre-computed)
- ✅ Production-ready modular code

## 👶**startign phase 0**

In [3]:
import kagglehub
path = kagglehub.dataset_download("olgaparfenova/daisee")


ModuleNotFoundError: No module named 'kagglehub'

In [ ]:
print("Downloaded to:", path)


In [ ]:
import shutil

src = path
dst = "/content/DAiSEE"

shutil.copytree(src, dst, dirs_exist_ok=True)


In [ ]:
import os

print(os.listdir("/content/DAiSEE"))
print(os.listdir("/content/DAiSEE/DAiSEE"))


In [ ]:
import os

print(os.listdir("/content/DAiSEE"))
print(os.listdir("/content/DAiSEE/DAiSEE"))


In [ ]:
import os

train_path = os.path.join(DATA_ROOT, "DataSet", "Train")

print("Train subjects:", len(os.listdir(train_path)))

first_subject = os.listdir(train_path)[0]
first_subject_path = os.path.join(train_path, first_subject)

print("Example subject:", first_subject)
print("Clips inside:", len(os.listdir(first_subject_path)))


In [ ]:
class DAiSEEDataset(Dataset):
    def __init__(self, root, split="Train", max_frames=30, frame_skip=5):
        self.root = root
        self.split = split
        self.max_frames = max_frames
        self.frame_skip = frame_skip
        
        self.video_root = os.path.join(root, "DataSet", split)
        self.labels = pd.read_csv(os.path.join(root, "Labels", f"{split}Labels.csv"))
        
        self.samples = []
        
        for _, row in self.labels.iterrows():
            clip_id = row["ClipID"]
            label = row["Engagement"]
            
            subject_id = clip_id[:6]
            clip_folder = clip_id.replace(".avi", "")
            
            video_path = os.path.join(
                self.video_root,
                subject_id,
                clip_folder,
                clip_id
            )
            
            if os.path.exists(video_path):
                self.samples.append((video_path, label))
        
        print(f"{split} samples loaded:", len(self.samples))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_path, label = self.samples[idx]
        
        cap = cv2.VideoCapture(video_path)
        frames = []
        frame_idx = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_idx % self.frame_skip == 0:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, (224, 224))
                frame = torch.tensor(frame).permute(2,0,1).float() / 255.0
                frames.append(frame)

            frame_idx += 1

            if len(frames) >= self.max_frames:
                break

        cap.release()

        if len(frames) == 0:
            frames = [torch.zeros(3, 224, 224)]

        # Padding
        if len(frames) < self.max_frames:
            pad = [torch.zeros(3,224,224)] * (self.max_frames - len(frames))
            frames.extend(pad)

        sequence = torch.stack(frames)  # (T, 3, 224, 224)

        return sequence, torch.tensor([label/3.0]).float()


In [ ]:
train_dataset = DAiSEEDataset(DATA_ROOT, split="Train")

train_loader = DataLoader(
    train_dataset,
    batch_size=4,   # small first
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

print("Dataset ready.")


## 📦 **1. Installation & Setup**

In [ ]:
# Install required packages
!pip install -q ultralytics transformers opencv-python scipy matplotlib seaborn tqdm
!pip install -q torch torchvision --upgrade

print("✅ All packages installed successfully!")

## 📚 **2. Imports**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import cv2
import numpy as np
from pathlib import Path
import json
from scipy.interpolate import BSpline

from transformers import ViTModel, ViTImageProcessor
from ultralytics import YOLO

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")

## 🎯 **3. YOLO Face Detector** (Kept from original)

In [ ]:
class YOLOFaceDetector:
    """
    YOLOv8-based real-time face detection.
    Used ONLY in Stage A (offline extraction).
    """

    def __init__(self, model_path='yolov8n.pt', conf_threshold=0.5, device='cuda'):
        self.model = YOLO(model_path)
        self.conf_threshold = conf_threshold
        self.device = device

    def detect_faces(self, frame, return_crops=True, padding=0.2):
        """
        Detect faces in frame.

        Args:
            frame: Input frame (H, W, 3) BGR
            return_crops: Return cropped faces
            padding: Padding around bbox (0.2 = 20%)

        Returns:
            List of detections with bboxes and crops
        """
        results = self.model(frame, conf=self.conf_threshold, verbose=False)

        detections = []

        for result in results:
            boxes = result.boxes

            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                conf = float(box.conf[0].cpu().numpy())

                detection = {
                    'bbox': [int(x1), int(y1), int(x2), int(y2)],
                    'confidence': conf
                }

                # Add padding
                if return_crops:
                    bbox_padded = self._add_padding(frame, detection['bbox'], padding)
                    detection['face_crop'] = self._crop_face(frame, bbox_padded)
                    detection['bbox_padded'] = bbox_padded

                detections.append(detection)

        return detections

    def _add_padding(self, frame, bbox, padding):
        x1, y1, x2, y2 = bbox
        h, w = frame.shape[:2]

        face_w = x2 - x1
        face_h = y2 - y1
        pad_w = int(face_w * padding)
        pad_h = int(face_h * padding)

        x1_pad = max(0, x1 - pad_w)
        y1_pad = max(0, y1 - pad_h)
        x2_pad = min(w, x2 + pad_w)
        y2_pad = min(h, y2 + pad_h)

        return [x1_pad, y1_pad, x2_pad, y2_pad]

    def _crop_face(self, frame, bbox):
        x1, y1, x2, y2 = bbox
        return frame[y1:y2, x1:x2].copy()

    def get_largest_face(self, detections):
        """Return largest face (primary student)"""
        if not detections:
            return None

        areas = [(d['bbox'][2] - d['bbox'][0]) * (d['bbox'][3] - d['bbox'][1]) for d in detections]
        return detections[np.argmax(areas)]


print("✅ YOLOFaceDetector class defined")

## 🧠 **4. Vision Transformer Feature Extractor** (Frozen)

In [ ]:
class ViTFeatureExtractor:
    """
    Vision Transformer for extracting facial features.
    FROZEN - no gradients during training.
    Used ONLY in Stage A (offline extraction).
    """

    def __init__(self, model_name='google/vit-base-patch16-224-in21k', device='cuda'):
        self.device = torch.device(device)

        print(f"Loading Vision Transformer: {model_name}")
        self.processor = ViTImageProcessor.from_pretrained(model_name)
        self.model = ViTModel.from_pretrained(model_name).to(self.device)
        self.model.eval()  # ALWAYS in eval mode

        # Freeze all parameters
        for param in self.model.parameters():
            param.requires_grad = False

        self.embedding_dim = self.model.config.hidden_size
        print(f"✅ ViT loaded (FROZEN) | Embedding dim: {self.embedding_dim}")

    def extract_features(self, face_image):
        """
        Extract features from face crop.

        Args:
            face_image: Face crop (H, W, 3) BGR or PIL Image

        Returns:
            features: (embedding_dim,) numpy array
        """
        # Convert BGR to RGB
        if isinstance(face_image, np.ndarray):
            face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
            face_image = Image.fromarray(face_image)

        # Preprocess
        inputs = self.processor(images=face_image, return_tensors="pt")
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        # Extract features (no gradients)
        with torch.no_grad():
            outputs = self.model(**inputs)
            # Use [CLS] token
            cls_token = outputs.last_hidden_state[:, 0, :].cpu().numpy()

        return cls_token.squeeze()


print("✅ ViTFeatureExtractor class defined")

## 🔬 **5. Kolmogorov-Arnold Network (KAN) Layer** (Kept from original)

In [ ]:
class KANLayer(nn.Module):
    """
    Kolmogorov-Arnold Network Layer with learnable B-spline basis functions.

    Unlike traditional neural networks with fixed activations,
    KAN learns the activation functions as B-splines.
    """

    def __init__(self, in_features, out_features, num_basis=8, spline_order=3, grid_range=(-1, 1)):
        super().__init__()

        self.in_features = in_features
        self.out_features = out_features
        self.num_basis = num_basis
        self.spline_order = spline_order
        self.grid_range = grid_range

        # Learnable spline coefficients
        self.spline_coeffs = nn.Parameter(
            torch.randn(in_features, out_features, num_basis) * 0.1
        )

        # Create B-spline knot vector
        num_knots = num_basis + spline_order + 1
        internal_knots = num_basis - spline_order + 1

        knots = np.concatenate([
            np.full(spline_order, grid_range[0]),
            np.linspace(grid_range[0], grid_range[1], internal_knots),
            np.full(spline_order, grid_range[1])
        ])

        self.register_buffer('knots', torch.tensor(knots, dtype=torch.float32))

    def forward(self, x):
        """
        Forward pass.

        Args:
            x: (batch_size, in_features)

        Returns:
            (batch_size, out_features)
        """
        # Normalize input to [-1, 1]
        x_normalized = torch.tanh(x)

        # Evaluate B-spline basis
        basis_values = self._evaluate_bspline_basis(x_normalized)

        # Apply coefficients
        output = torch.einsum('bik,iok->bo', basis_values, self.spline_coeffs)

        return output

    def _evaluate_bspline_basis(self, x):
        """
        Evaluate B-spline basis functions.

        Args:
            x: (batch_size, in_features) in [-1, 1]

        Returns:
            (batch_size, in_features, num_basis)
        """
        batch_size, in_features = x.shape
        device = x.device

        # Initialize basis matrix
        basis = torch.zeros(batch_size, in_features, self.num_basis, device=device)

        # Clamp to grid range
        x_clamped = torch.clamp(x, self.grid_range[0], self.grid_range[1])

        # Simplified polynomial basis
        for k in range(self.num_basis):
            basis[:, :, k] = x_clamped ** k

        # Normalize
        basis = F.normalize(basis, p=2, dim=2)

        return basis


print("✅ KANLayer class defined")

---
# 🔵 **STAGE A: OFFLINE FEATURE EXTRACTION**
---

**This section runs ONCE to extract embeddings from videos.**

## 📹 **6. Video Embedding Extraction Function**

In [ ]:
def extract_video_embeddings(video_path, yolo_detector, vit_extractor,
                              max_frames=30, frame_skip=5):
    """
    Extract temporal embeddings from a video.

    Args:
        video_path: Path to video file
        yolo_detector: YOLOFaceDetector instance
        vit_extractor: ViTFeatureExtractor instance
        max_frames: Maximum frames to extract
        frame_skip: Sample every N frames

    Returns:
        embeddings: numpy array (max_frames, embedding_dim)
    """
    cap = cv2.VideoCapture(video_path)
    embeddings = []
    frame_idx = 0
    frames_extracted = 0

    while frames_extracted < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        # Sample frames
        if frame_idx % frame_skip == 0:
            # Detect face
            detections = yolo_detector.detect_faces(frame, return_crops=True)

            if detections:
                # Get largest face
                face_data = yolo_detector.get_largest_face(detections)
                face_crop = face_data['face_crop']

                # Extract ViT features
                features = vit_extractor.extract_features(face_crop)
                embeddings.append(features)
                frames_extracted += 1

        frame_idx += 1

    cap.release()

    # Handle empty sequences
    if len(embeddings) == 0:
        print(f"WARNING: No faces detected in {video_path}")
        embeddings = [np.zeros(vit_extractor.embedding_dim)]

    # Convert to array
    sequence = np.array(embeddings)

    # Pad to max_frames
    if len(sequence) < max_frames:
        pad_size = max_frames - len(sequence)
        pad = np.zeros((pad_size, vit_extractor.embedding_dim))
        sequence = np.vstack([sequence, pad])
    else:
        sequence = sequence[:max_frames]  # Truncate if too long

    return sequence  # Shape: (max_frames, embedding_dim)


print("✅ extract_video_embeddings function defined")

## 🚀 **7. Batch Extraction Pipeline**

In [ ]:
def extract_all_embeddings(video_paths, labels, yolo_detector, vit_extractor,
                            save_dir='/content/daisee_embeddings',
                            max_frames=30, frame_skip=5):
    """
    Extract embeddings for all videos and save to disk.

    Args:
        video_paths: List of video file paths
        labels: List of corresponding labels
        yolo_detector: YOLO detector
        vit_extractor: ViT extractor
        save_dir: Directory to save embeddings
        max_frames: Max frames per video
        frame_skip: Frame sampling rate

    Returns:
        metadata: List of dicts with paths and labels
    """
    # Create save directory
    save_path = Path(save_dir)
    save_path.mkdir(parents=True, exist_ok=True)

    metadata = []

    print(f"Extracting embeddings for {len(video_paths)} videos...")

    for idx, (video_path, label) in enumerate(tqdm(zip(video_paths, labels),
                                                     total=len(video_paths),
                                                     desc="Extracting")):
        # Extract embeddings
        embeddings = extract_video_embeddings(
            video_path, yolo_detector, vit_extractor,
            max_frames=max_frames, frame_skip=frame_skip
        )

        # Generate unique ID
        video_id = Path(video_path).stem
        embedding_file = save_path / f"{video_id}.npy"

        # Save embeddings
        np.save(embedding_file, embeddings)

        # Store metadata
        metadata.append({
            'video_id': video_id,
            'embedding_path': str(embedding_file),
            'label': label,
            'shape': embeddings.shape
        })

    # Save metadata
    metadata_file = save_path / 'metadata.json'
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=2)

    print(f"\n✅ Extraction complete!")
    print(f"   Saved {len(metadata)} embedding files to: {save_dir}")
    print(f"   Metadata saved to: {metadata_file}")

    return metadata


print("✅ extract_all_embeddings function defined")

## 🎬 **8. Run Stage A: Extract Embeddings**

**⚠️ IMPORTANT: Only run this cell ONCE after collecting your video dataset.**

In [ ]:
# Initialize YOLO and ViT for extraction
print("Initializing YOLO and ViT for Stage A...")
yolo_detector = YOLOFaceDetector(device=device)
vit_extractor = ViTFeatureExtractor(device=device)

print("\n" + "="*60)
print("STAGE A: OFFLINE FEATURE EXTRACTION")
print("="*60)

# TODO: Replace with your actual video paths and labels
# Example:
# video_paths = [
#     '/content/videos/student1_confused.mp4',
#     '/content/videos/student2_not_confused.mp4',
#     # ...
# ]
# labels = [1.0, 0.0, ...]  # 1.0 = confused, 0.0 = not confused

# For demo purposes:
print("\nTo run extraction:")
print("1. Prepare your video dataset")
print("2. Create lists: video_paths and labels")
print("3. Run: metadata = extract_all_embeddings(video_paths, labels, yolo_detector, vit_extractor)")
print("4. This will save .npy files to /content/daisee_embeddings/")
print("\nExample code:")
print("""
# video_paths = glob.glob('/content/daisee_videos/*.mp4')
# labels = [get_label(v) for v in video_paths]  # Your label extraction logic
# metadata = extract_all_embeddings(video_paths, labels, yolo_detector, vit_extractor)
""")

# Uncomment when ready:
# metadata = extract_all_embeddings(video_paths, labels, yolo_detector, vit_extractor)

---
# 🟢 **STAGE B: TRAINING & INFERENCE**
---

**This section trains on pre-extracted embeddings.**

## 📦 **9. Lightweight Embedding Dataset**

In [ ]:
class EmbeddingDataset(Dataset):
    """
    Lightweight dataset that loads pre-extracted embeddings.

    NO video I/O. NO YOLO. NO ViT.
    Just loads .npy files.
    """

    def __init__(self, metadata_or_paths, embedding_dir=None):
        """
        Args:
            metadata_or_paths: Either:
                - List of dicts from extract_all_embeddings
                - Path to metadata.json file
            embedding_dir: Directory containing .npy files (if using json path)
        """
        if isinstance(metadata_or_paths, (str, Path)):
            # Load from JSON
            with open(metadata_or_paths, 'r') as f:
                self.metadata = json.load(f)
        else:
            self.metadata = metadata_or_paths

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        """
        Returns:
            embeddings: Tensor (max_frames, embedding_dim)
            label: Tensor (1,)
        """
        item = self.metadata[idx]

        # Load embeddings from disk
        embeddings = np.load(item['embedding_path'])

        # Convert to tensors
        embeddings_tensor = torch.tensor(embeddings, dtype=torch.float32)
        label_tensor = torch.tensor([item['label']], dtype=torch.float32)

        return embeddings_tensor, label_tensor


print("✅ EmbeddingDataset class defined")

## 🧠 **10. LSTM + KAN Confusion Model**

In [ ]:
class ConfusionModel(nn.Module):
    """
    Complete confusion detection model.

    Architecture:
        Embeddings (T, 768) → LSTM → KAN → Confusion Score (0-1)
    """

    def __init__(self, input_dim=768, lstm_hidden=256, lstm_layers=2,
                 kan_hidden_dims=[128, 64], dropout=0.3,
                 num_basis=8, spline_order=3):
        super().__init__()

        self.input_dim = input_dim
        self.lstm_hidden = lstm_hidden

        # LSTM for temporal modeling
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=lstm_hidden,
            num_layers=lstm_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if lstm_layers > 1 else 0
        )

        lstm_output_dim = lstm_hidden * 2  # Bidirectional

        # KAN layers
        self.kan_layers = nn.ModuleList()

        prev_dim = lstm_output_dim
        for hidden_dim in kan_hidden_dims:
            self.kan_layers.append(
                KANLayer(prev_dim, hidden_dim, num_basis, spline_order)
            )
            prev_dim = hidden_dim

        # Final KAN output layer
        self.kan_output = KANLayer(prev_dim, 1, num_basis, spline_order)

        # Dropout
        self.dropout = nn.Dropout(dropout)

        # Output activation
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        """
        Forward pass.

        Args:
            x: (batch, seq_len, 768) embeddings

        Returns:
            confusion_score: (batch, 1) in [0, 1]
        """
        # LSTM encoding
        lstm_out, _ = self.lstm(x)  # (batch, seq_len, lstm_hidden*2)

        # Take last timestep
        lstm_last = lstm_out[:, -1, :]  # (batch, lstm_hidden*2)

        # KAN layers
        x = lstm_last
        for kan_layer in self.kan_layers:
            x = kan_layer(x)
            x = self.dropout(x)

        # Output
        x = self.kan_output(x)
        confusion_score = self.sigmoid(x)

        return confusion_score


print("✅ ConfusionModel class defined")

## 🏋️ **11. Training Loop**

In [ ]:
def train_confusion_model(model, train_loader, val_loader,
                           num_epochs=20, learning_rate=0.001,
                           loss_type='bce', device='cuda',
                           save_path='best_confusion_model.pth'):
    """
    Train confusion detection model.

    Args:
        model: ConfusionModel instance
        train_loader: Training DataLoader
        val_loader: Validation DataLoader
        num_epochs: Number of epochs
        learning_rate: Learning rate
        loss_type: 'bce' or 'mse'
        device: Device to train on
        save_path: Path to save best model

    Returns:
        model: Trained model
        history: Training history dict
    """
    model = model.to(device)

    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Loss function
    if loss_type == 'bce':
        criterion = nn.BCELoss()
    elif loss_type == 'mse':
        criterion = nn.MSELoss()
    else:
        raise ValueError(f"Unknown loss_type: {loss_type}")

    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=3, verbose=True
    )

    # Training history
    history = {
        'train_loss': [],
        'val_loss': [],
        'val_accuracy': []
    }

    best_val_loss = float('inf')

    print("\n" + "="*60)
    print("TRAINING START")
    print("="*60)

    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0.0

        train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
        for sequences, labels in train_bar:
            sequences = sequences.to(device)
            labels = labels.to(device)

            # Forward
            predictions = model(sequences)
            loss = criterion(predictions, labels)

            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_bar.set_postfix({'loss': f"{loss.item():.4f}"})

        train_loss /= len(train_loader)

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")
            for sequences, labels in val_bar:
                sequences = sequences.to(device)
                labels = labels.to(device)

                predictions = model(sequences)
                loss = criterion(predictions, labels)

                val_loss += loss.item()

                # Accuracy
                pred_labels = (predictions > 0.5).float()
                correct += (pred_labels == labels).sum().item()
                total += labels.size(0)

        val_loss /= len(val_loader)
        val_accuracy = correct / total

        # Update history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_accuracy)

        # Learning rate scheduling
        scheduler.step(val_loss)

        # Print stats
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Val Loss:   {val_loss:.4f}")
        print(f"  Val Acc:    {val_accuracy:.4f}")

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
                'val_accuracy': val_accuracy
            }, save_path)
            print(f"  ✅ Best model saved! (val_loss: {val_loss:.4f})")

        print()

    print("="*60)
    print("TRAINING COMPLETE")
    print("="*60)

    return model, history


print("✅ train_confusion_model function defined")

## 🚀 **12. Run Stage B: Training**

In [ ]:
# Create datasets (assumes embeddings already extracted)
print("Creating datasets from pre-extracted embeddings...")

# Load metadata
embedding_dir = Path('/content/daisee_embeddings')
metadata_file = embedding_dir / 'metadata.json'

if not metadata_file.exists():
    print("⚠️ Metadata file not found! Run Stage A first.")
    print(f"Expected location: {metadata_file}")
else:
    # Load metadata
    with open(metadata_file, 'r') as f:
        all_metadata = json.load(f)

    # Split into train/val (80/20)
    split_idx = int(0.8 * len(all_metadata))
    train_metadata = all_metadata[:split_idx]
    val_metadata = all_metadata[split_idx:]

    print(f"Train samples: {len(train_metadata)}")
    print(f"Val samples: {len(val_metadata)}")

    # Create datasets
    train_dataset = EmbeddingDataset(train_metadata)
    val_dataset = EmbeddingDataset(val_metadata)

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)

    print("\n✅ Datasets ready!")

    # Initialize model
    print("\nInitializing ConfusionModel...")
    confusion_model = ConfusionModel(
        input_dim=768,
        lstm_hidden=256,
        lstm_layers=2,
        kan_hidden_dims=[128, 64],
        dropout=0.3
    ).to(device)

    # Model summary
    total_params = sum(p.numel() for p in confusion_model.parameters())
    trainable_params = sum(p.numel() for p in confusion_model.parameters() if p.requires_grad)
    print(f"\nModel Parameters:")
    print(f"  Total: {total_params:,}")
    print(f"  Trainable: {trainable_params:,}")

    # Train
    print("\nStarting training...")
    trained_model, history = train_confusion_model(
        model=confusion_model,
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=20,
        learning_rate=0.001,
        loss_type='bce',
        device=device,
        save_path='best_confusion_model.pth'
    )

## 📊 **13. Plot Training History**

In [ ]:
def plot_training_history(history):
    """Plot training curves."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Loss curves
    ax1.plot(history['train_loss'], label='Train Loss', linewidth=2)
    ax1.plot(history['val_loss'], label='Val Loss', linewidth=2)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training & Validation Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Accuracy curve
    ax2.plot(history['val_accuracy'], label='Val Accuracy', linewidth=2, color='green')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Validation Accuracy')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
    plt.show()

    print("✅ Training curves saved to training_curves.png")


# Plot if history exists
try:
    plot_training_history(history)
except NameError:
    print("Run training first to generate history")

## 🎯 **14. Inference Function**

In [ ]:
def predict_from_sequence(model, sequence, device='cuda'):
    """
    Run inference on a single embedding sequence.

    Args:
        model: Trained ConfusionModel
        sequence: numpy array (T, 768) or tensor
        device: Device

    Returns:
        confusion_score: float in [0, 1]
    """
    model.eval()

    # Convert to tensor if needed
    if isinstance(sequence, np.ndarray):
        sequence = torch.tensor(sequence, dtype=torch.float32)

    # Add batch dimension
    sequence = sequence.unsqueeze(0).to(device)  # (1, T, 768)

    # Inference
    with torch.no_grad():
        score = model(sequence).item()

    return score


def predict_from_embedding_file(model, embedding_path, device='cuda'):
    """
    Run inference on a saved embedding file.

    Args:
        model: Trained ConfusionModel
        embedding_path: Path to .npy file
        device: Device

    Returns:
        confusion_score: float in [0, 1]
    """
    embeddings = np.load(embedding_path)
    score = predict_from_sequence(model, embeddings, device)
    return score


print("✅ Inference functions defined")

## 🧪 **15. Test Inference**

In [ ]:
# Test inference on a sample
try:
    # Load best model
    checkpoint = torch.load('best_confusion_model.pth', map_location=device)
    confusion_model.load_state_dict(checkpoint['model_state_dict'])
    print("✅ Best model loaded")
    print(f"   Val Loss: {checkpoint['val_loss']:.4f}")
    print(f"   Val Accuracy: {checkpoint['val_accuracy']:.4f}")

    # Test on a random embedding file
    embedding_files = list(embedding_dir.glob('*.npy'))
    if embedding_files:
        test_file = embedding_files[0]
        print(f"\nTesting on: {test_file.name}")

        score = predict_from_embedding_file(confusion_model, test_file, device)
        print(f"Predicted confusion score: {score:.4f}")

        if score > 0.7:
            print("→ High confusion detected! ⚠️")
        elif score > 0.4:
            print("→ Moderate confusion")
        else:
            print("→ Low confusion ✅")
    else:
        print("No embedding files found for testing")

except FileNotFoundError:
    print("Model checkpoint not found. Train the model first.")

## 💾 **16. Save/Load Model Utilities**

In [ ]:
def save_complete_model(model, save_path='edusense_complete.pth'):
    """
    Save model with full config for easy loading.
    """
    torch.save({
        'model_state_dict': model.state_dict(),
        'config': {
            'input_dim': model.input_dim,
            'lstm_hidden': model.lstm_hidden,
        }
    }, save_path)
    print(f"✅ Complete model saved to: {save_path}")


def load_complete_model(load_path='edusense_complete.pth', device='cuda'):
    """
    Load model with automatic config restoration.
    """
    checkpoint = torch.load(load_path, map_location=device)

    # Recreate model
    model = ConfusionModel(
        input_dim=checkpoint['config']['input_dim'],
        lstm_hidden=checkpoint['config']['lstm_hidden']
    ).to(device)

    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    print(f"✅ Model loaded from: {load_path}")
    return model


# Save current model
try:
    save_complete_model(confusion_model, 'edusense_lstm_kan.pth')
except NameError:
    print("Train model first before saving")

---
# 🎓 **Summary**
---

## ✅ What We Built

### **Stage A: Offline Feature Extraction**
- YOLOv8 face detection
- ViT feature extraction (frozen)
- Saved embeddings to `.npy` files
- **Run once per video dataset**

### **Stage B: Training & Inference**
- Lightweight `EmbeddingDataset` (no video I/O)
- LSTM for temporal modeling
- KAN for interpretable classification
- Fast training on pre-extracted features

## 🔑 Key Advantages

1. **Decoupled Architecture**
   - Feature extraction separate from training
   - Easy to experiment with different models

2. **Efficiency**
   - No video processing during training
   - ViT frozen (no backprop through transformer)
   - 10-100x faster training iterations

3. **Production-Ready**
   - Embeddings can be pre-computed in batch jobs
   - Lightweight inference (just LSTM + KAN)
   - Easy deployment

4. **Modular**
   - Can swap LSTM for GRU/Transformer
   - Can add more KAN layers
   - Can use different ViT models

## 📂 File Structure

```
/content/daisee_embeddings/
├── video1.npy          # (30, 768) embeddings
├── video2.npy
├── ...
└── metadata.json       # Labels and paths

Models:
├── best_confusion_model.pth      # Best checkpoint
├── edusense_lstm_kan.pth         # Final model
└── training_curves.png           # Visualizations
```

## 🚀 Next Steps

1. ✅ Extract embeddings from your DAiSEE dataset
2. ✅ Train LSTM + KAN model
3. Fine-tune hyperparameters (LSTM layers, KAN hidden dims)
4. Add attention mechanism if needed
5. Deploy for real-time confusion detection

---

**King Khalid University**  
**EduSense Graduation Project 2025**  
**Production-Ready 2-Stage Architecture ✅**